<a href="https://colab.research.google.com/github/aarish7771/Computer_Vision_Models/blob/main/DCGANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Convoluted Generative Adversarial Networks (DCGANs)

###Importing Libraries

In [12]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

###Setting Hyperparameters

In [13]:
batchSize = 64
imageSize = 64

###Creating the Transformations

In [14]:
transform = transforms.Compose([transforms.Resize((imageSize, imageSize)), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])

###Loading the Dataset

In [15]:
dataset = dset.CIFAR10(root = './data', download = True, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2)

Files already downloaded and verified


###Defining the weights_init function that takes as input a neural network m and that will initialize all its weights

In [16]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

### Defining the Generator

In [17]:
class G(nn.Module):

  def __init__(self):
    super(G, self).__init__()
    self.main = nn.Sequential(
        nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
        nn.BatchNorm2d(512),
        nn.ReLU(True),
        nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),
        nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
        nn.BatchNorm2d(128),
        nn.ReLU(True),
        nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
        nn.Tanh()
        )

  def forward(self, input):
    output = self.main(input)
    return output

### Creating the Generator

In [18]:
NetG = G()
NetG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

### Defining the Discriminator

In [19]:
class D(nn.Module):

  def __init__(self):
    super(D, self).__init__()
    self.main = nn.Sequential(
        nn.Conv2d(3, 64, 4, 2, 1, bias = False),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(64, 128, 4, 2, 1, bias = False),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(128, 256, 4, 2, 1, bias = False),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(256, 512, 4, 2, 1, bias = False),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(512, 1, 4, 1, 0, bias = False),
        nn.Sigmoid()
        )

  def forward(self, input):
    output = self.main(input)
    return output.view(-1)

### Creating the Discriminator

In [20]:
NetD = D()
NetD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

### Training the DCGANs

In [ ]:
criterion = nn.BCELoss()
optimizerD = optim.Adam(NetD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(NetG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

for epoch in range(25):
  for i, data in enumerate(dataloader, 0):

    # 1st Step: Updating the weights of the Neural Network of the Discriminator

    NetD.zero_grad()

    # Training the Discriminator with the real images of the Dataset
    real, _ = data
    input = Variable(real)
    target = Variable(torch.ones(input.size()[0]))
    output = NetD(input)
    errD_real = criterion(output, target)

    # Traininig the Discriminator with the fake images generated by the Generator
    noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
    fake = NetG(noise)
    target = Variable(torch.zeros(input.size()[0]))
    output = NetD(fake.detach())
    errD_fake = criterion(output, target)

    # Back-propagating the total error
    errD = errD_real + errD_fake
    errD.backward()
    optimizerD.step()

    # 2nd Step: Updating the weights of the Neural Network of the Generator

    NetG.zero_grad()
    target = Variable(torch.ones(input.size()[0]))
    output = NetD(fake)
    errG = criterion(output, target)
    errG.backward()
    optimizerG.step()

    # 3rd Step: Printing the losses and saving the Real and Generated Images of the Minibatch at every 100th step
    print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD, errG))
    if i % 100 == 0:
      vutils.save_image(real, '%s/real_samples.png' % "./Results", normalize = True)
      fake = NetG(noise)
      vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./Results", epoch), normalize = True)



[0/25][0/782] Loss_D: 1.7160 Loss_G: 7.2397
[0/25][1/782] Loss_D: 1.0190 Loss_G: 4.8311
[0/25][2/782] Loss_D: 1.1474 Loss_G: 5.0672
[0/25][3/782] Loss_D: 1.2330 Loss_G: 6.3412
[0/25][4/782] Loss_D: 0.4788 Loss_G: 6.7453
[0/25][5/782] Loss_D: 0.5957 Loss_G: 6.4565
[0/25][6/782] Loss_D: 0.9307 Loss_G: 7.7577
[0/25][7/782] Loss_D: 0.8932 Loss_G: 8.0372
[0/25][8/782] Loss_D: 0.7526 Loss_G: 7.5883
[0/25][9/782] Loss_D: 0.6696 Loss_G: 9.7054
[0/25][10/782] Loss_D: 0.4784 Loss_G: 7.2766
[0/25][11/782] Loss_D: 1.1149 Loss_G: 11.7501
[0/25][12/782] Loss_D: 0.7721 Loss_G: 9.0866
[0/25][13/782] Loss_D: 0.6800 Loss_G: 8.9936
[0/25][14/782] Loss_D: 0.7050 Loss_G: 12.5333
[0/25][15/782] Loss_D: 0.2976 Loss_G: 9.9467
[0/25][16/782] Loss_D: 0.5430 Loss_G: 11.6576
[0/25][17/782] Loss_D: 0.3139 Loss_G: 8.8241
[0/25][18/782] Loss_D: 0.6820 Loss_G: 11.6179
[0/25][19/782] Loss_D: 0.1414 Loss_G: 9.4662
[0/25][20/782] Loss_D: 0.4259 Loss_G: 11.9910
[0/25][21/782] Loss_D: 0.1104 Loss_G: 9.8044
[0/25][22/782] 